In [ ]:
# Market Risk Analysis — Historical VaR & CVaR

This notebook contains the implementation of a historical market risk analysis for a diversified ETF portfolio.
Detailed objectives, methodology, and discussion are provided in the project README.


In [ ]:
## Environment setup and configuration

import pandas as pd
import numpy as np
import yfinance as yf

pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)


In [ ]:
## Portfolio definition and assumptions

TICKERS = ["SPY", "QQQ", "IWM", "TLT", "GLD", "HYG"]

WEIGHTS = {
    "SPY": 0.20,
    "QQQ": 0.20,
    "IWM": 0.15,
    "TLT": 0.20,
    "GLD": 0.15,
    "HYG": 0.10
}

START_DATE = "2015-01-01"
END_DATE = None  
sum(WEIGHTS.values())


In [ ]:
## Data retrieval
def stooq_prices(ticker):
    url = f"https://stooq.com/q/d/l/?s={ticker.lower()}.us&i=d"
    df = pd.read_csv(url)
    df["Date"] = pd.to_datetime(df["Date"])
    df = df[df["Date"] >= START_DATE]
    df = df[["Date", "Close"]].rename(columns={"Close": "adj_close"})
    df["ticker"] = ticker
    return df

prices = (
    pd.concat([stooq_prices(t) for t in TICKERS], ignore_index=True)
      .rename(columns={"Date": "date"})
      .sort_values(["ticker", "date"])
      .reset_index(drop=True)
)

prices.head(), prices.shape


In [ ]:
## Data cleaning and validation

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

prices_path = data_dir / "prices.csv"
prices.to_csv(prices_path, index=False)

prices_path.resolve()


In [ ]:
## Daily return calculation

prices["return"] = (
    prices
    .groupby("ticker")["adj_close"]
    .pct_change()
)

returns = prices.dropna().copy()
returns.head()


In [ ]:
from pathlib import Path

data_dir = Path("data")
data_dir.mkdir(exist_ok=True)

prices_path = data_dir / "prices.csv"
prices.to_csv(prices_path, index=False)

prices_path.resolve()


In [ ]:
returns.groupby("ticker").size()


In [ ]:
prices.shape


In [ ]:
# Ensure correct types + sorting
prices = prices.copy()
prices["date"] = pd.to_datetime(prices["date"])
prices = prices.sort_values(["ticker", "date"]).reset_index(drop=True)

# Daily simple returns per ticker
prices["ret"] = prices.groupby("ticker")["adj_close"].pct_change()

returns = prices.dropna(subset=["ret"]).copy()
returns.head()


In [ ]:
## Exploratory return statistics

returns.groupby("ticker")["ret"].agg(["count", "mean", "std"])


In [ ]:
## Portfolio return construction

WEIGHTS = {
    "SPY": 0.20,
    "QQQ": 0.20,
    "IWM": 0.15,
    "TLT": 0.20,
    "GLD": 0.15,
    "HYG": 0.10
}

returns["weight"] = returns["ticker"].map(WEIGHTS)

# Check if any ticker failed to map a weight
returns["weight"].isna().sum()


In [ ]:
returns["w_ret"] = returns["ret"] * returns["weight"]

portfolio = (
    returns.groupby("date")["w_ret"].sum()
    .rename("portfolio_ret")
    .to_frame()
    .sort_index()
)

portfolio.head()


In [ ]:
## Portfolio loss distribution

portfolio["loss"] = -portfolio["portfolio_ret"]
portfolio.describe()


In [ ]:
## Historical VaR and CVaR estimation

def historical_var_cvar(losses: pd.Series, alpha: float):
    var = losses.quantile(alpha)
    cvar = losses[losses >= var].mean()
    return float(var), float(cvar)

for a in [0.95, 0.99]:
    v, c = historical_var_cvar(portfolio["loss"], a)
    print(f"Historical {int(a*100)}%  VaR: {v:.4%}   CVaR: {c:.4%}")
